# New Section

In [ ]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/noisy_test_data.txt

--2024-11-15 17:15:34--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375849 (367K) [text/plain]
Saving to: ‘train_data.txt’

train_data.txt      100%[===================>] 367.04K  --.-KB/s    in 0.009s  

2024-11-15 17:15:34 (39.1 MB/s) - ‘train_data.txt’ saved [375849/375849]

--2024-11-15 17:15:34--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP reques

In [ ]:
import numpy as np
from collections import defaultdict

# Read data from files
def read_data(filename):
    sentences = []
    with open(filename, 'r') as f:
        for line in f:
            sentences.append([tuple(word.split('/')) for word in line.strip().split()])
    return sentences

# Load training, test, and noisy test data
train_data = read_data('/content/train_data.txt')
test_data = read_data('/content/test_data.txt')
noisy_test_data = read_data('/content/noisy_test_data.txt')


In [ ]:


#Derive hidden and observable states from training data and building Hidden Markovnikov Model
def build_hmm(train_data):
    transition_probs = defaultdict(lambda: defaultdict(int))
    emission_probs = defaultdict(lambda: defaultdict(int))
    tag_counts = defaultdict(int)

    for sentence in train_data:
        prev_tag = '<START>'
        for item in sentence:
            if len(item) != 2:
                continue  # Skip elements that don't have exactly (word, tag) format

            word, tag = item
            transition_probs[prev_tag][tag] += 1
            emission_probs[tag][word] += 1
            tag_counts[tag] += 1
            prev_tag = tag
        transition_probs[prev_tag]['<END>'] += 1

    # Normalize probabilities
    transition_probs = {
        k: {k2: v2 / sum(v.values()) for k2, v2 in v.items()}
        for k, v in transition_probs.items()
    }
    emission_probs = {
        k: {k2: v2 / tag_counts[k] for k2, v2 in v.items()}
        for k, v in emission_probs.items()
    }

    return transition_probs, emission_probs, list(tag_counts.keys())

# Build HMM
transition_probs, emission_probs, tags = build_hmm(train_data)


# Viterbi Algorithm
def viterbi(sentence, tags, transition_probs, emission_probs):
    n = len(sentence)
    m = len(tags)
    dp = np.zeros((n, m))
    backpointer = np.zeros((n, m), dtype=int)

    # Initialize
    for j, tag in enumerate(tags):
        dp[0, j] = transition_probs['<START>'].get(tag, 0) * emission_probs[tag].get(sentence[0], 1e-6)

    # Fill DP Table
    for i in range(1, n):
        for j, tag in enumerate(tags):
            max_prob = -1
            best_prev_tag = -1
            for k, prev_tag in enumerate(tags):
                prob = dp[i-1, k] * transition_probs[prev_tag].get(tag, 1e-6) * emission_probs[tag].get(sentence[i], 1e-6)
                if prob > max_prob:
                    max_prob = prob
                    best_prev_tag = k
            dp[i, j] = max_prob
            backpointer[i, j] = best_prev_tag

    # Termination
    best_path = []
    max_final_prob = -1
    best_final_tag = -1
    for j, tag in enumerate(tags):
        prob = dp[-1, j] * transition_probs[tag].get('<END>', 1e-6)
        if prob > max_final_prob:
            max_final_prob = prob
            best_final_tag = j

    # Backtrace
    current_tag = best_final_tag
    for i in range(n-1, -1, -1):
        best_path.insert(0, tags[current_tag])
        current_tag = backpointer[i, current_tag]

    return best_path


# Handle Noisy Data
# Adjust emission probabilities for unknown words
def handle_noise(emission_probs, unseen_penalty=1e-6):
    for tag in emission_probs.keys():
        emission_probs[tag]['<UNK>'] = unseen_penalty
    return emission_probs

# Adjust emission probabilities to handle noise
emission_probs_noisy = handle_noise(emission_probs)


# Evaluation Function
def evaluate(test_data, tags, transition_probs, emission_probs):
    correct, total = 0, 0
    for sentence in test_data:
        # Separate words and actual tags from the sentence tuple
        words, true_tags = zip(*sentence)

        # Predict tags using Viterbi
        predicted_tags = viterbi(words, tags, transition_probs, emission_probs)

        # Calculate the number of correct predictions
        correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
        total += len(true_tags)

    # Calculate accuracy
    accuracy = correct / total
    return accuracy

# Evaluate baseline Viterbi on the test data
baseline_accuracy = evaluate(test_data, tags, transition_probs, emission_probs)

# Evaluate noise-handled Viterbi on the noisy test data
noise_handled_accuracy = evaluate(noisy_test_data, tags, transition_probs, emission_probs_noisy)

# Output Results
print(f"Baseline Viterbi Accuracy: {baseline_accuracy * 100:.2f}%")
print(f"Noise-handled Viterbi Accuracy: {noise_handled_accuracy * 100:.2f}%")


Baseline Viterbi Accuracy: 90.18%
Noise-handled Viterbi Accuracy: 81.77%
